# Exact solution of a linear acoustic propagation PDE.

Implementing a function that calculates the pressure field at time $t>0$ at cylindrical coordinates $(r,\theta)$. 

We let the upper integration boundary of the solution formula be a variable argument $\Omega>0$ which is supposed to approach infinity to approximate the improper integral (1).

In [1]:
using QuadGK
using SpecialFunctions

#function to calculate the integrand, which would then be used for quadgk.
function pulse_int(t, r, theta, w)
    M = 0.5
    b = log(2)/(w^2)
    integrand(w) = A / (2*b) * exp(-w^2 / (4*b)) * besselj(0,w*sqrt(r^2 + (M*t)^2 - 2r*(M*t)*cos(theta))) * w * cos(w*t)
    return integrand
end

#function to calculate the whole integral.
function pulse_tot(t, r, theta, w, omega)
    M = 0.5
    b = log(2)/(w^2)
    integrand(w) = A / (2 * b) * exp(-w^2 / (4 * b)) * besselj(0, w * sqrt(r^2 + (M * t)^2 - 2 * r * (M * t) * cos(theta))) * w * cos(w * t)
    return quadgk(integrand, 0, omega)[1]
end

pulse_tot (generic function with 1 method)

We now let the amplitude of the pulse be $A=0.001$ and its half-width be $w=3.0$. And we test our function at $t=1,r=1,\theta=0$ and different upper integration boundaries $\Omega>0$. 

Next, we find the smallest value of $\Omega$ such that the computed value of our function does not change by more than $0.01\%$ by changing from $\Omega$ to $2\Omega$ and we compare this value with the exact value at $\Omega=\infty$, which can be achieved by using the argument $Inf$ as upper boundary in 'quadgk'.

In [ ]:
#parameters
A = 0.001
w = 3.0
t = 1.0
r = 1.0
theta = 0.0
b = log(2) / (w^2)

omega = 0.1
initial_omega = 0.000001

prev = pulse_tot(t, r, theta, w, initial_omega)
new = prev

#while loop to find the smallest desirable omega.
while true
    new = pulse_tot(t, r, theta, w, omega)

    if (abs(new - prev) / prev) < 0.0001
        break
    end

    prev = new
    omega *= 2
end

exact_value = pulse_tot(t, r, theta, w, Inf)

println("Smallest Omega: ", omega)
println("Computed Value: ", new)
println("Exact Value: ", exact_value)

Smallest Omega: 3.2
Computed Value: 0.000839984001921987
Exact Value: 0.0008399840019219854


In [3]:
#obtaining the exact value with inf at the upper limit.
omega_val = pulse_tot(t, r, theta, w, omega)
exact = pulse_tot(t, r, theta, w, Inf)

println("Value: ", omega_val)
println("Exact value: ", exact)

Value: 0.000839984001921987
Exact value: 0.0008399840019219854


Now, we test our function at $t=1,r=1000,\theta=0$ and $\Omega=\infty$ in two different ways: First, by setting the relative tolerance argument (rtol) in  'quadgk' to 1e-7 and, second, by setting the absolute tolerance argument (atol) in 'quadgk' to 1e-7.

In [4]:
#parameters
A = 0.001
w = 3.0
t = 1.0
r = 1000.0 # this makes some difference.
theta = 0.0

Omega = Inf # upper limit

#results with different tolerances (relative & absolute)
result1 = quadgk(pulse_int(t, r, theta, w), 0, Omega, rtol=1e-7)[1]
result2 = quadgk(pulse_int(t, r, theta, w), 0, Omega, atol=1e-7)[1]

println("Result with rtol=1e-7: ", result1)
println("Result with atol=1e-7: ", result2)

Result with rtol=1e-7: -9.458559888538118e-22
Result with atol=1e-7: -1.6464404249962117e-6


We evaluate our function at two different times $t_1=500\Delta t, t_2=1150\Delta t$, where $\Delta t =0.0569$, and we plot the pressure profile along the $x$ axis from $0$ to $100$, i.e. at $\theta=0,r\in(0,100)$. 

Let's also compare our results with Fig. 11 from the following article: http://acoustics.ae.illinois.edu/pdfs/tam-drp-1993.pdf.

In [ ]:
using Plots, LaTeXStrings
import Printf
using QuadGK

#parameters
theta = 0
xs = 2.0
A = 1.0
w = 3.0
M = 0.5

dt = 0.0569
t1 = 500dt
t2 = 1150dt

Omega = Inf #upper limit

#range and number of points
num_points = 75
r = range(-100.0, 100.0, length=num_points)

#plotting the pressure profile at both t_1 and t_2
for t in (t1, t2)
    pressure_profile = zeros(num_points)
    for i in 1:num_points
        pressure_profile[i] = pulse_tot(t, r[i], theta, w, Omega)
    end
    plt = plot(r, pressure_profile, xlabel="x", ylabel="Pressure", label="t=$(t/dt)", linecolor=:black)
    savefig(plt, "./figures/profile/pressure_t$(t/dt).png")
end

The exact solution for the pressure field of the configuration in the previous part is given by
\begin{equation}
 p(r,\theta,t) =  \int_{0}^{\infty} \mathcal{H}\{\phi'_0\}(\omega) \left(  J_0\left( \omega \sqrt{r^2+ x_s^2-2r\cdot x_s \cdot \cos(\theta)} + \right) -  J_0\left( \omega \sqrt{r^2+ x_s^2+2r\cdot x_s  \cos(\theta)} + \right) \right)\cdot \omega \cos(\omega t)   d\omega \ \ \ \ \ \ \ \ \   (3)
\end{equation}
where $\mathcal{H}\{\phi'_0\}(\omega)$ is given by expression (2). 

For $x_s=1.0$, we implement a function that evaluates $p$ at all times $t>0$ and locations $(r,\theta)$. Let the pulse amplitude be $A=1.0$ and inverse pulse width be $b=2$. We also verify that our function computes the correct result on the $y$ axis as determined in the previous part.

In [8]:
using QuadGK
using SpecialFunctions

#parameters
A = 1.0
b = 2.0
xs = 1.0

function pressure_field(t, r, theta, A, b, xs)
    hankel(w) = A / (2 * b) * exp(-w^2 / (4 * b))
    integrand(w) = H(w, A, b) * (besselj(0, w * sqrt(r^2 + xs^2 - 2 * r * xs * cos(theta))) - besselj(0, w * sqrt(r^2 + xs^2 + 2 * r * xs * cos(theta)))) * w * cos(w * t)
    p, _ = quadgk(integrand, 0, Inf)
    return p
end

num_points = 11
t_values = range(0, 11, num_points)

r = xs
theta = pi / 2

pressure_val = zeros(num_points)

#evaluating the function at different t's.
for t in t_values
    pressure_val = pressure_field(t, r, theta, A, b, xs)
    println("Pressure at (r,θ) = ($(r), $(theta)) and t = $(t) is: ", pressure_val)
end

Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 0.0 is: 2.2733866226186134e-17


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 1.1 is: 2.3713797774720285e-17


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 2.2 is: -2.205494212028711e-17


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 3.3 is: 6.012691233952722e-18


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 4.4 is: 2.3863486238049883e-18


...


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 6.6 is: 1.9576907457112998e-19


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 7.7 is: 1.1848770431361342e-18


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 8.8 is: -1.0743664802742749e-19


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 9.9 is: -3.223860530212107e-19


Pressure at (r,θ) = (1.0, 1.5707963267948966) and t = 11.0 is: 4.627947909527966e-19


We are indeed getting near-zero values for points in the y-axis!

For the same parameters, let's create an animation of the pressure field on the $x$ axis ranging from $0$ to $5$ for times $0<t<7$.

In [6]:
using Plots, LaTeXStrings, QuadGK, SpecialFunctions
import Printf

#parameters
x0 = 0
xs = 1.0
b = 2.0

Nx = 21
Ny = 21
w = 3.0

Nt = 10
t0 = 0.000
t1 = 7.0

c = 0.05
A = 1.0

function pressure_field(t, r, theta, A, b, xs)
    hankel(w) = A / (2 * b) * exp(-w^2 / (4 * b))
    integrand(w) = hankel(w) * (besselj(0, w * sqrt(r^2 + xs^2 - 2 * r * xs * theta)) - besselj(0, w * sqrt(r^2 + xs^2 + 2 * r * xs * theta))) * w * cos(w * t)
    p, _ = quadgk(integrand, 0, Inf)
    return p
end

pressure_field (generic function with 1 method)

In [ ]:
#spatial and time grid
x = range(0.001, 5.0, Nx)
y = range(-3, 3, Ny)
t = LinRange(t0, t1, Nt)

for i in 1:Nt

    time = t[i]

    f(x, y) = begin
        r = sqrt((x + c * time)^2 + y^2)
        pressure_field(time, r, x/r, A, b, xs)
    end

    plt = Plots.contour(x, y, f, fill = true, c=:cividis)
    plt = Plots.title!(plt, "Time = " * string(time) * " seconds")
    Plots.png(plt, "./figures/anim_figs/part-H/$(lpad(i, 6, "0")).png")
end

fnames = [Printf.@sprintf("./figures/anim_figs/part-H/%06d.png", i) for i in 1:Nt]
an = Animation("./figures/anim_figs/part-H", fnames)
Plots.buildanimation(an, "./figures/animation/Animation_pressure_field.gif", fps = 1)

In [21]:
#calculating the time the original pulse hits the y-axis and the time it returns to the location (xs, 0).
max_y = -Inf
max_xs = -Inf
time_y = 0
time_xs = 0
new_t = LinRange(t0, t1, Nt)

zero_xs = Inf
time_zero_xs = 0

nonzero_xs = Inf
time_nonzero_xs = 0

for i in 1:Nt
    t = new_t[i]
    p_y = pressure_field(t, r, π/2, A, b, xs)
    p_xs = pressure_field(t, r, 0, A, b, xs)

    if p_y > max_y
        max_y = p_y
        time_y = t
    end

    if p_xs > max_xs
        max_xs = p_xs
        time_xs = t
    end

    #time p_xs becomes zero for the first time
    if p_xs <= 0 && t < zero_xs
        zero_xs = t
        time_zero_xs = t
    end

    #time p_xs becomes non-zero again
    if p_xs > 0 && t > zero_xs && t < nonzero_xs
        nonzero_xs = t
        time_nonzero_xs = t
    end
end

#in case p_xs never becomes zero
if time_zero_xs == 0
    time_zero_xs = Inf
    time_nonzero_xs = Inf
end

In [22]:

println("The pulse hits the y-axis at: ", time_y)
println("The pulse returns to (x_s, 0) at: ", time_nonzero_xs)

The pulse hits the y-axis at: 0.75
The pulse return to (x_s, 0) at: 2.5


Let's also generate a gif-animation of the pressure field from $t=0$ to $t=8$ on the domain $(x,y) \in (\epsilon, 3.4)\times (-2.2,2.2)$, where $\epsilon=0.001$. Use 65 equidistant points to resolve the $x$ and $y$ dimensions, and 33 equidistant time steps.

In [ ]:
#parameters
Nx=65;
Ny=65;

Nt=33
t0 = 0.000
t1 = 8.0

c=0.05
ϵ=0.001

x=range(ϵ, 3.4, length=Nx)
y=range(-2.2, 2.2, length=Ny)
t=LinRange(t0, t1, Nt)

for i =1:Nt
    time=t[i]

    f(x, y) = begin
        r = sqrt((x + c * time)^2 + y^2)
        pressure_field(time, r, x/r, A, b, xs)
    end

    Plots.contour(x, y, f, fill=true, c=:cividis)
    plt=Plots.title!("Time = " * string(time) * " seconds")
    Plots.png(plt,"./figures/anim_figs/bonus-I/$(lpad(i,6,"0")).png")
end

fnames=[Printf.@sprintf("./figures/anim_figs/bonus-I/%06d.png", i) for i =1:Nt]

an=Animation("./figures/anim_figs/bonus-I/", fnames)
Plots.buildanimation(an,"./figures/animation/Animation-bonusI.gif", fps=1)

Now, we are creating a multi-threaded variant of our function and also generating a high-resolution gif-animation of the pressure field from $t=0$ to $t=8$ on the domain $(x,y) \in (\epsilon, 3.4)\times (-2.2,2.2)$, where $\epsilon=0.001$. 

We use 129 equidistant points to resolve the $x$ and $y$ dimensions, and 81 equidistant time steps.

Note: Ran with 4 cores.

In [ ]:
using Base.Threads

#parameters
Nx = 129
Ny = 129

Nt = 81
t0 = 0.000
t1 = 8.0

x = range(0.001, 3.4, Nx)
y = range(-2.2, 2.2, Ny)
t = LinRange(t0, t1, Nt)

#multi-threaded variant of the function.
function pressure_field_mt(Nx, Ny, Nt, x, y, times, c, theta, A, b, xs)
    pressure_values = Array{Float64}(undef, Nx, Ny, Nt)

    #looping through time steps in parallel.
    @threads for i in 1:Nt
        time = times[i]

        #looping through x and y points in parallel.
        Threads.@threads for j in 1:Nx
            for k in 1:Ny
                r = sqrt((x[j] + c * time)^2 + y[k]^2)
                pressure_values[k, j, i] = pressure_field(time, r, x[j]/r, A, b, xs)
            end
        end
    end

    return pressure_values
end

pressure_values = pressure_field_mt(Nx, Ny, Nt, x, y, times, c, theta, A, b, xs)

for i in 1:Nt
    plt = Plots.contour(x, y, pressure_values[:, :, i], fill = true, c=:cividis)
    plt = Plots.title!(plt, "Time = " * string(times[i]) * " seconds")
    Plots.png(plt, "./figures/anim_figs/bonus-J/$(lpad(i, 6, "0")).png")
end

fnames = [Printf.@sprintf("./figures/anim_figs/bonus-J/%06d.png", i) for i in 1:Nt]
an = Animation("./figures/anim_figs/bonus-J/", fnames)
Plots.buildanimation(an, "./figures/animation/Animation_bonusJ.gif", fps = 3)